In [1]:
import re
import string
import pandas as pd
from math import log
import os
import random
import re
import string
from collections import Counter
from sklearn.cluster import KMeans
import pickle

import nltk
import numpy as np
import pandas as pd

from gensim.models import Word2Vec

from nltk import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings("ignore")
nltk.download("stopwords")


SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

[nltk_data] Downloading package stopwords to /Users/joker/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
final_counts = []

In [3]:
def clean_text(text):
    text = str(text).lower() 
    text = text.translate (str.maketrans('', '', string.punctuation))
    text = re.sub("\d+", "", text)
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub('[!()-[]{};:\'"\,<>./?@#$%^&*_~–]+', '', text)
    return text

In [8]:
all_roman = [] #list to keep all the grievances with more Devnagri words than non-Devnagri


In [5]:
pref = "/Volumes/olaaaa/Thesis_backup/data/"

In [9]:
for file in os.listdir("/Volumes/olaaaa/Thesis_backup/data"):
    #print(file, "here")
    if file.endswith("csv"):
        # print("here")
        df = pd.read_csv(pref+file, header = None, low_memory=False)
        # print(len(df))
        df = df[[0,21]]
        df.columns = ['reg_no', 'subject_content']
        
        df['subject_content_cleaned'] = df['subject_content'].apply(clean_text)
        tmp = [ val for val in df['subject_content_cleaned'].values \
        if len(re.findall("[a-z]+",val)) > 0.5*len(val.split())]
        all_roman.extend(tmp) 
        print(file, "Size of updated list", str(len(all_roman)))
        # print()
        
        
        

chunk8.csv Size of updated list 771813
chunk9.csv Size of updated list 1086405
chunk2.csv Size of updated list 1887324
chunk3.csv Size of updated list 2817237
chunk1.csv Size of updated list 3714626
chunk0.csv Size of updated list 4583761
chunk4.csv Size of updated list 5579426
chunk5.csv Size of updated list 6536852
chunk7.csv Size of updated list 7270426
chunk6.csv Size of updated list 8053182


In [10]:
len(all_roman)

8053182

In [11]:

with open('all_roman.pkl', 'wb') as f:
    pickle.dump(all_roman, f)

In [12]:
with open('all_roman.pkl', 'rb') as f:
    loaded = pickle.load(f)

In [ ]:
df = pd.DataFrame(all_roman)

In [ ]:
df.columns = ['subject_content']

In [18]:
def freq(sent):
    arr = np.zeros(26)
    for letter in sent:
        if 97 <= ord(letter) <= 122:
            # print("here")
            arr[ord(letter) - 97] += 1 
    #arr[::-1].sort()
    
    return arr / np.linalg.norm(arr) #normalization

In [17]:
# list(df.subject_content.values)

In [20]:
df['histograms'] = df['subject_content'].apply(freq)

In [21]:
df.isna().sum()

subject_content    0
isDevnagri         0
histograms         0
dtype: int64

In [22]:
df_non_roman_non_devnagri.head()

,subject_content,isDevnagri,histograms
13,respected sirmadam আমার নাম জয়ন্ত মণ্ডল আমার ...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
30,மதிப்பிற்குரிய ஐயா வணக்கம் பொருள் எரிவாயு சப்ள...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
40,নমস্কার স্যার আমি আপনার কাছে একটাই অনুরোধ যে আ...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
52,ঘার কে লিয়ে এপ্লাই,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
58,স্যার আমি একজন গরীব কৃষক কুলতলী থানার কাওড়াখাল...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [23]:
with open('df_non_roman_non_devnagri_histograms.pkl', 'wb') as f:
    pickle.dump(df_non_roman_non_devnagri, f)

In [42]:
with open('df_non_roman_non_devnagri_histograms.pkl', 'rb') as f:
    df = pickle.load(f)

In [43]:
clusters = 7
def mbkmeans_clusters(X, k):
    km = KMeans(n_clusters=k).fit(X)
    return km.labels_

In [44]:
df

,subject_content,isDevnagri,histograms
13,respected sirmadam আমার নাম জয়ন্ত মণ্ডল আমার ...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
30,மதிப்பிற்குரிய ஐயா வணக்கம் பொருள் எரிவாயு சப்ள...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
40,নমস্কার স্যার আমি আপনার কাছে একটাই অনুরোধ যে আ...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
52,ঘার কে লিয়ে এপ্লাই,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
58,স্যার আমি একজন গরীব কৃষক কুলতলী থানার কাওড়াখাল...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...
1349791,માનનીય સાહેબ જીતના અભિનંદન પણ દાહોદ ની જીતથી હ...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1349793,மேதகு பாரதபிரதமர் அவர்களுக்கு பணிவான முதல் வணக...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1349808,মোৰ শ্ৰদ্ধাৰ মৰমৰ আদৰণীয় সন্মানীয় মাননীয় ভাৰতৰ...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1349822,பாரத் மாதாகி ஜெய் பாரத் மாதாகி ஜெய் பாரத் மாதா...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [45]:
df = df.loc[ df.histograms.map(lambda x: ~np.isnan(x).any()), :]


In [46]:
df['labels'] = mbkmeans_clusters(
    X=list(df['histograms'].values),
    k=clusters,
)

In [47]:
df.labels.value_counts()

1    58914
2    11580
0     8950
4     5335
5     3223
3     1101
6      891
Name: labels, dtype: int64

In [41]:
df[df.labels == 2].subject_content.to_csv("/Volumes/olaaaa/Thesis_backup/count_languages_devnagri/scripts/"+str(2)+".csv")

In [48]:
for i in range(clusters):
    print(df[df.labels == i])
    df[df.labels == i].subject_content.to_csv("/Volumes/olaaaa/Thesis_backup/count_languages_devnagri/scripts/"+str(i)+".csv")

                                           subject_content isDevnagri  \
101      ಭಾರತ ಮತ್ತು ಚೀನಾ ನಡುವಿನ ಯುದ್ಧದಲ್ಲಿ ನನಗೆ ಭಾಗವಹಿಸ...              
109      ಭಾರತ ಮತ್ತು ಚೀನಾ ನಡುವಿನ ಯುದ್ಧದಲ್ಲಿ ನನಗೆ ಭಾಗವಹಿಸ...              
270      dear sir i m ganesh kundapura ಸರ್ ಭಾರತ ಚೀನಾದ ಗ...              
617      ಭಾರತದ ಪ್ರಧಾನ ಮಂತ್ರಿಗಳಾದ ಶ್ರೀ ನರೇಂದ್ರ ದಾಮೋದರದಾಸ...              
840      ಸರ್ ನನ್ನದೊಂದು ವಿನಂತಿ ನಾನು sansera engineering ...              
...                                                    ...        ...   
1349512  dear sir ಯಾವ ಧರ್ಮವನ್ನು ದ್ವೇಷಿಸಲಿಲ್ಲ ಯಾವ ಜಾತಿಯನ...              
1349608  ಬಿಜೆಪಿ ಅಭೂತಪೂವ ಗೆಲುವಿನಲ್ಲಿ ಪ್ರಮುಖ ಪಾತ್ರ ವಹಿಸಿದ...              
1349689   ಲೋಕಸಭಾ ಚುಣಾವನೆಯಲ್ಲಿ ಮುನ್ನಡೆ ಸಾಧಿಸಿರುವ ನನ್ನ ಆತ...              
1349712  ಪ್ರಧಾನ ಮಂತ್ರಿಗಳು ಶ್ರೀ ಮೋದಿಯವರಿಗೆ ಹೃದಯಪೂರ್ವಕ ಜಯ...              
1349721  ಪ್ರಧಾನ ಮಂತ್ರಿ ಗಳಲ್ಲಿ ಮನವಿ ಸಮುದ್ರಕ್ಕೆ ಹತ್ತಿರವಿರ...              

                                                histograms  labels  
101      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,